# Install OpenAI Agents SDK

In [14]:
!pip install -Uq openai-agents

# Make your Notebook capable of running asynchronous functions.


In [15]:
import nest_asyncio
nest_asyncio.apply()

In [67]:
import asyncio
from agents import Agent, handoff, Runner, RunContextWrapper, set_tracing_disabled, enable_verbose_stdout_logging

# config

In [69]:
from pydantic import BaseModel
from agents import (
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    RunConfig)
from google.colab import userdata

In [70]:
set_tracing_disabled(True)
enable_verbose_stdout_logging()

gemini_api_key = userdata.get("GEMINI_API_KEY")


# Check if the API is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")


#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)

# Panacloud Agent

In [71]:
_devops_agent = Agent(
    name="DevOps Agent",
    instructions="Answer questions about DevOps",
    handoff_description="An assitant to answer Devops related queries.",
    model=model)

_openai_agent = Agent(
    name="OpenAI Agent",
    instructions="Answer questions about OpenAI Agents SDK",
    handoff_description="An assistant to answer OpenAI Agent SDK related queries",
    model=model)

In [72]:
web_dev_agent = Agent(
    name="Web Dev Agent",
    instructions="Answer questions about web development",)

mobile_dev_agent = Agent(
    name="Mobile Dev Agent",
    instructions="Answer questions about mobile development",)

agentic_ai_agent = Agent(
    name="Agentic AI Agent",
    instructions=(
        "You are an Agentic AI Specialist. You use the tools given to you to to answer questions."
        "If you asked multiple questions, you call the relevant tools in order."
        "You never answer on your own, always call the provided tools."
    ),
    tools=[_devops_agent.as_tool(
        tool_name="devops_specilist",
        tool_description="Answer questions about DevOps"),
          _openai_agent.as_tool(
        tool_name="openai_sdk_specialist",
        tool_description="Answer questions about OpenAI"),
          ]
    )

In [73]:
def on_handoff(agent: Agent, ctx: RunContextWrapper[None]):
    agent_name = agent.name
    print("--------------------------------")
    print(f"Handing off to {agent_name}...")
    print("--------------------------------")




panacloud_agent = Agent(
    name="Panacloud Agent",
    instructions="Handoff to the appropriate agent based on the query",
    handoffs=[handoff(web_dev_agent, on_handoff= lambda ctx: on_handoff(web_dev_agent, ctx)),
              handoff(mobile_dev_agent, on_handoff= lambda ctx: on_handoff(mobile_dev_agent, ctx)),
              handoff(agentic_ai_agent, on_handoff= lambda ctx: on_handoff(agentic_ai_agent, ctx))]
)


In [74]:
async def main(input: str):
  result = await Runner.run(panacloud_agent, input=input, run_config=config)
  print(result.final_output)

In [75]:
asyncio.run(main("Which stack is popular in 2025 for Mobile app development?"))

Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op
Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x7ed829cf1250>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x7ed829cf1250>


...

LLM resp:
{
  "content": "Okay, let's speculate on the popular mobile app development stacks in 2025. It's impossible to say with 100% certainty, but we can make some educated guesses based on current trends and emerging technologies:\n\n**Factors Influencing Stack Popularity in 2025:**\n\n*   **Cross-Platform Dominance:** The need to target both iOS and Android efficiently will continue to drive the popularity of cross-platform frameworks.\n*   **Performance:** Native-like performance will always be a key consideration. Frameworks that can deliver near-native performance will be favored.\n*   **Developer Productivity:** Tools and frameworks that allow developers to build apps quickly and efficiently will be in high demand.\n*   **AI Integration:** As AI becomes more prevalent, frameworks that facilitate the integration of AI features (e.g., machine learning models) will gain traction.\n*   **WebAssembly (Wasm):** Wasm could play a bigger role, allowing for portable and high-performanc

DEBUG:openai.agents:LLM resp:
{
  "content": "Okay, let's speculate on the popular mobile app development stacks in 2025. It's impossible to say with 100% certainty, but we can make some educated guesses based on current trends and emerging technologies:\n\n**Factors Influencing Stack Popularity in 2025:**\n\n*   **Cross-Platform Dominance:** The need to target both iOS and Android efficiently will continue to drive the popularity of cross-platform frameworks.\n*   **Performance:** Native-like performance will always be a key consideration. Frameworks that can deliver near-native performance will be favored.\n*   **Developer Productivity:** Tools and frameworks that allow developers to build apps quickly and efficiently will be in high demand.\n*   **AI Integration:** As AI becomes more prevalent, frameworks that facilitate the integration of AI features (e.g., machine learning models) will gain traction.\n*   **WebAssembly (Wasm):** Wasm could play a bigger role, allowing for portable

Resetting current trace
Resetting current trace


DEBUG:openai.agents:Resetting current trace


Okay, let's speculate on the popular mobile app development stacks in 2025. It's impossible to say with 100% certainty, but we can make some educated guesses based on current trends and emerging technologies:

**Factors Influencing Stack Popularity in 2025:**

*   **Cross-Platform Dominance:** The need to target both iOS and Android efficiently will continue to drive the popularity of cross-platform frameworks.
*   **Performance:** Native-like performance will always be a key consideration. Frameworks that can deliver near-native performance will be favored.
*   **Developer Productivity:** Tools and frameworks that allow developers to build apps quickly and efficiently will be in high demand.
*   **AI Integration:** As AI becomes more prevalent, frameworks that facilitate the integration of AI features (e.g., machine learning models) will gain traction.
*   **WebAssembly (Wasm):** Wasm could play a bigger role, allowing for portable and high-performance code across platforms.
*   **Mat

In [77]:
asyncio.run(main("Tell me about latest top 5 Web development frameworks?"))

Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op
Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x7ed7f7ce2b10>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x7ed7f7ce2b10>


...

LLM resp:
{
  "content": "Okay, here's a rundown of 5 top web development frameworks that are currently popular and considered strong choices, along with some of their key features and use cases:\n\n**Keep in Mind:**  \"Top\" is subjective and depends heavily on the project's specific needs, team expertise, and development philosophy.  This list represents a blend of popularity, community support, performance, and modern features.  Also, the web development landscape changes *very* rapidly, so this is current as of late 2024.\n\n1.  **React (JavaScript Library/Framework)**\n\n    *   **What it is:**  React is a JavaScript library (often used as a framework with tooling) for building user interfaces (UIs). It's maintained by Facebook and a large community.\n    *   **Key Features:**\n        *   **Component-Based Architecture:**  UIs are broken down into reusable components, making development modular and maintainable.\n        *   **Virtual DOM:**  React uses a virtual DOM to efficient

DEBUG:openai.agents:LLM resp:
{
  "content": "Okay, here's a rundown of 5 top web development frameworks that are currently popular and considered strong choices, along with some of their key features and use cases:\n\n**Keep in Mind:**  \"Top\" is subjective and depends heavily on the project's specific needs, team expertise, and development philosophy.  This list represents a blend of popularity, community support, performance, and modern features.  Also, the web development landscape changes *very* rapidly, so this is current as of late 2024.\n\n1.  **React (JavaScript Library/Framework)**\n\n    *   **What it is:**  React is a JavaScript library (often used as a framework with tooling) for building user interfaces (UIs). It's maintained by Facebook and a large community.\n    *   **Key Features:**\n        *   **Component-Based Architecture:**  UIs are broken down into reusable components, making development modular and maintainable.\n        *   **Virtual DOM:**  React uses a virt

Resetting current trace
Resetting current trace


DEBUG:openai.agents:Resetting current trace


Okay, here's a rundown of 5 top web development frameworks that are currently popular and considered strong choices, along with some of their key features and use cases:

**Keep in Mind:**  "Top" is subjective and depends heavily on the project's specific needs, team expertise, and development philosophy.  This list represents a blend of popularity, community support, performance, and modern features.  Also, the web development landscape changes *very* rapidly, so this is current as of late 2024.

1.  **React (JavaScript Library/Framework)**

    *   **What it is:**  React is a JavaScript library (often used as a framework with tooling) for building user interfaces (UIs). It's maintained by Facebook and a large community.
    *   **Key Features:**
        *   **Component-Based Architecture:**  UIs are broken down into reusable components, making development modular and maintainable.
        *   **Virtual DOM:**  React uses a virtual DOM to efficiently update only the necessary parts of

In [76]:
asyncio.run(main("How do you see Agentic AI imapact on Devops Market, Is the Devops maket going to increase? Call the tool available tool."))

Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op
Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x7ed7f7ce2810>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x7ed7f7ce2810>


...

LLM resp:
{
  "content": "Yes, the DevOps market is likely to increase with the emergence of Agentic AI. Agentic AI enhances automation, reduces operational overhead, and improves efficiency. This will drive wider adoption of DevOps practices and create new market opportunities.\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}

LLM resp:
{
  "content": "Yes, the DevOps market is likely to increase with the emergence of Agentic AI. Agentic AI enhances automation, reduces operational overhead, and improves efficiency. This will drive wider adoption of DevOps practices and create new market opportunities.\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



DEBUG:openai.agents:LLM resp:
{
  "content": "Yes, the DevOps market is likely to increase with the emergence of Agentic AI. Agentic AI enhances automation, reduces operational overhead, and improves efficiency. This will drive wider adoption of DevOps practices and create new market opportunities.\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



Resetting current trace
Resetting current trace


DEBUG:openai.agents:Resetting current trace


Yes, the DevOps market is likely to increase with the emergence of Agentic AI. Agentic AI enhances automation, reduces operational overhead, and improves efficiency. This will drive wider adoption of DevOps practices and create new market opportunities.

